# Customer Loyalty Preprocessing

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import time
from datetime import datetime 

In [3]:
import warnings
warnings.filterwarnings("ignore")

## Import/Optimize Datasets

In [4]:
def get_memory_size(dataframe):
    return dataframe.memory_usage(deep=True).sum() / (1024 ** 2)

def optimize_memory_usage(dataframe):
    optimized_df = pd.DataFrame()

    for col in dataframe.columns:
        original_col = dataframe[col]
        if original_col.dtypes == 'int':
            optimized_col = pd.to_numeric(original_col, downcast='integer')

        elif original_col.dtypes == 'float':
            optimized_col = pd.to_numeric(original_col, downcast='float')

        elif original_col.dtypes == 'object':
            if len(original_col) / len(original_col.unique()) > 0.5:
                optimized_col = original_col.astype('category')

            else:
                optimized_col = original_col
        else:
            optimized_col = original_col

        optimized_df[col] = optimized_col

    return optimized_df

def get_reduced_df(dataframe_chunks):
  optimized_dfs = []

  for chunk in dataframe_chunks:
      optimized_df_chunk = optimize_memory_usage(chunk)
      optimized_dfs.append(optimized_df_chunk)

  return pd.concat(optimized_dfs)

In [5]:
chunk_size = 20000

historical_transactions_chunks = pd.read_csv('./Data/historical_transactions.csv', chunksize=chunk_size)
merchants_chunks = pd.read_csv('./Data/merchants.csv', chunksize=chunk_size)
new_merchant_transactions_chunks = pd.read_csv('./Data/new_merchant_transactions.csv', chunksize=chunk_size)
train_chunks = pd.read_csv('./Data/train.csv', chunksize=chunk_size)
test_chunks = pd.read_csv('./Data/test.csv', chunksize=chunk_size)

start_time = time.time()
historical_transactions = get_reduced_df(historical_transactions_chunks)
del historical_transactions_chunks
print("time to load historical_transactions {} seconds".format(int(time.time() - start_time)))

start_time = time.time()
merchants = get_reduced_df(merchants_chunks)
del merchants_chunks
print("time to load merchants {} seconds".format(int(time.time() - start_time)))

start_time = time.time()
new_merchant_transactions = get_reduced_df(new_merchant_transactions_chunks)
del new_merchant_transactions_chunks
print("time to load new_merchant_transactions {} seconds".format(int(time.time() - start_time)))

start_time = time.time()
train = get_reduced_df(train_chunks)
del train_chunks
print("time to load train {} seconds".format(int(time.time() - start_time)))

start_time = time.time()
test = get_reduced_df(test_chunks)
del test_chunks
print("time to load test {} seconds".format(int(time.time() - start_time)))

time to load historical_transactions 102 seconds
time to load merchants 1 seconds
time to load new_merchant_transactions 6 seconds
time to load train 0 seconds
time to load test 0 seconds


## Merging/Preprocessing Transaction Histories

In [6]:
# Remember which entries came from which dataset
historical_transactions['new_merchant_flag'] = False
new_merchant_transactions['new_merchant_flag'] = True

In [7]:
transactions = pd.concat([historical_transactions, new_merchant_transactions], ignore_index=True)
transactions.shape

(31075392, 15)

In [8]:
transactions.isna().sum()

authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
category_3               234081
merchant_category_id          0
merchant_id              164697
month_lag                     0
purchase_amount               0
purchase_date                 0
category_2              2764609
state_id                      0
subsector_id                  0
new_merchant_flag             0
dtype: int64

In [9]:
# Without merchant_id, entries lose quite a bit of associated information in merchants.csv
transactions.dropna(subset=['merchant_id'], inplace=True)

In [10]:
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])

In [11]:
# Create separate features for year/month/day/hour
transactions['year'] = transactions['purchase_date'].dt.year.astype('int16')
transactions['month'] = transactions['purchase_date'].dt.month.astype('int16')
transactions['day'] = transactions['purchase_date'].dt.day.astype('int16')
transactions['hour'] = transactions['purchase_date'].dt.hour.astype('int16')

In [12]:
# Taking a look at the data before encoding
transactions.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,new_merchant_flag,year,month,day,hour
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,False,2017,6,25,15
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,False,2017,7,15,12
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,False,2017,8,9,22


In [13]:
# One hot encode categorical features
ohe_features = [
    'authorized_flag',
    'new_merchant_flag',
    'category_1',
    'category_2',
    'category_3'
]

# Maintain a column for NA values after OHE
transactions['category_2'] = transactions['category_2'].astype('category')
transactions['category_2'] = transactions['category_2'].cat.add_categories('NA')
transactions['category_2'].fillna('NA', inplace=True)

transactions['category_3'] = transactions['category_3'].astype('category')
transactions['category_3'] = transactions['category_3'].cat.add_categories('NA')
transactions['category_3'].fillna('NA', inplace=True)

# Do the encoding
encoded_df = pd.get_dummies(transactions, columns=ohe_features, prefix=ohe_features, drop_first=True)

In [14]:
# Any remaining NaN values? (no)
encoded_df.isna().sum()

card_id                   0
city_id                   0
installments              0
merchant_category_id      0
merchant_id               0
month_lag                 0
purchase_amount           0
purchase_date             0
state_id                  0
subsector_id              0
year                      0
month                     0
day                       0
hour                      0
authorized_flag_Y         0
new_merchant_flag_True    0
category_1_Y              0
category_2_2.0            0
category_2_3.0            0
category_2_4.0            0
category_2_5.0            0
category_2_NA             0
category_3_B              0
category_3_C              0
category_3_NA             0
dtype: int64

In [15]:
# Look at the datatypes
encoded_df.dtypes

card_id                           object
city_id                            int64
installments                       int64
merchant_category_id               int64
merchant_id                       object
month_lag                          int64
purchase_amount                  float64
purchase_date             datetime64[ns]
state_id                           int64
subsector_id                       int64
year                               int16
month                              int16
day                                int16
hour                               int16
authorized_flag_Y                   bool
new_merchant_flag_True              bool
category_1_Y                        bool
category_2_2.0                      bool
category_2_3.0                      bool
category_2_4.0                      bool
category_2_5.0                      bool
category_2_NA                       bool
category_3_B                        bool
category_3_C                        bool
category_3_NA   

In [16]:
# Check the ranges of int64 features
int64_features = ['city_id', 'installments', 'merchant_category_id', 'month_lag', 'state_id', 'subsector_id']

for feature in int64_features:
    print(feature, "\t:\t", encoded_df[feature].min(), " ", encoded_df[feature].max())


city_id 	:	 -1   347
installments 	:	 -1   999
merchant_category_id 	:	 -1   891
month_lag 	:	 -13   2
state_id 	:	 -1   24
subsector_id 	:	 -1   41


In [17]:
# Above features do not need 64 bits, reduce them to 16
for feature in int64_features:
    encoded_df[feature] = encoded_df[feature].astype(np.int16)

In [18]:
# Before optimizing, the data consumed is the following.
# Note: this is AFTER the initial optimization of the incoming .csv files -- before any optimization, there is ~14 Gb
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 30910695 entries, 0 to 31075391
Data columns (total 19 columns):
 #   Column                Dtype         
---  ------                -----         
 0   authorized_flag       object        
 1   card_id               object        
 2   city_id               int64         
 3   category_1            category      
 4   installments          int64         
 5   category_3            category      
 6   merchant_category_id  int64         
 7   merchant_id           object        
 8   month_lag             int64         
 9   purchase_amount       float64       
 10  purchase_date         datetime64[ns]
 11  category_2            category      
 12  state_id              int64         
 13  subsector_id          int64         
 14  new_merchant_flag     bool          
 15  year                  int16         
 16  month                 int16         
 17  day                   int16         
 18  hour                  int16         
dtypes: 

In [19]:
encoded_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 30910695 entries, 0 to 31075391
Data columns (total 25 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   card_id                 object        
 1   city_id                 int16         
 2   installments            int16         
 3   merchant_category_id    int16         
 4   merchant_id             object        
 5   month_lag               int16         
 6   purchase_amount         float64       
 7   purchase_date           datetime64[ns]
 8   state_id                int16         
 9   subsector_id            int16         
 10  year                    int16         
 11  month                   int16         
 12  day                     int16         
 13  hour                    int16         
 14  authorized_flag_Y       bool          
 15  new_merchant_flag_True  bool          
 16  category_1_Y            bool          
 17  category_2_2.0          bool          
 18  categ

In [20]:
renaming = {
    'authorized_flag_Y' : 'authorized_flag',
    'new_merchant_flag_True' : 'new_merchant_flag',
    'category_1_Y' : 'category_1'
}

encoded_df.rename(columns=renaming, inplace=True)

In [21]:
encoded_df.head(3)

,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,...,new_merchant_flag,category_1,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_NA,category_3_B,category_3_C,category_3_NA
0,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,...,False,False,False,False,False,False,False,False,False,False
1,C_ID_4e6213e9bc,88,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,...,False,False,False,False,False,False,False,False,False,False
2,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,...,False,False,False,False,False,False,False,False,False,False


In [22]:
encoded_df.to_csv('./Data/encoded_df.csv', index=False)
encoded_df.head(3)

,card_id,city_id,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,...,new_merchant_flag,category_1,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_NA,category_3_B,category_3_C,category_3_NA
0,C_ID_4e6213e9bc,88,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,...,False,False,False,False,False,False,False,False,False,False
1,C_ID_4e6213e9bc,88,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,...,False,False,False,False,False,False,False,False,False,False
2,C_ID_4e6213e9bc,88,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,...,False,False,False,False,False,False,False,False,False,False
